## Installation (work on colab)

In [ ]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf
!pip install gym

!pip install free-mujoco-py
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install imageio==2.4.1
!pip install -U colabgymrender
!pip install mujoco

In [ ]:
!pip install pfrl

In [3]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 10.5 MB/s eta 0:00:00


In [4]:
from typing import Optional, List
from copy import deepcopy

import functools
import numpy as np
from sklearn.utils import check_random_state

import gymnasium as gym
import gym.wrappers
from gym.spaces import Discrete

import pfrl

In [5]:
device = "cpu"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
%cd /content/drive/MyDrive/codes

/content/drive/.shortcut-targets-by-id/1f55zVGF_dF_ofwI_JfGgeIj4XSrHgcE8/codes


# Original env

In [8]:
# ctx = torch.multiprocessing.get_context("spawn")

NameError: ignored

In [ ]:
# torch.multiprocessing.set_start_method('spawn')

RuntimeError: ignored

In [9]:
env_name = "Hopper-v3"
num_envs = 1
seed = 42

monitor = False
render = False

process_seeds = np.arange(num_envs) + seed * num_envs

In [10]:
def make_env(process_idx, test):
    env = gym.make(env_name)  #
    # env = gym.make(env_name, exclude_current_positions_from_observation=False)  #
    # Unwrap TimiLimit wrapper
    assert isinstance(env, gym.wrappers.TimeLimit)
    env = env.env
    # Use different random seeds for train and test envs
    process_seed = int(process_seeds[process_idx])
    env_seed = 2**32 - 1 - process_seed if test else process_seed
    env.seed(env_seed)
    # Cast observations to float32 because our model uses float32
    env = pfrl.wrappers.CastObservationToFloat32(env)
    # Normalize action space to [-1, 1]^n
    env = pfrl.wrappers.NormalizeActionSpace(env)
    if monitor:
        env = gym.wrappers.Monitor(env, "monitor")
    if render:
        env = pfrl.wrappers.Render(env)
    return env

In [12]:
env = make_env(0, test=False)

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment Hopper-v3 is out of date. You should consider upgrading to version `v4`.
  logger.warn(


Compiling /usr/local/lib/python3.10/dist-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /usr/local/lib/python3.10/dist-packages/mujoco_py/cymj.pyx


INFO:root:running build_ext
INFO:root:building 'mujoco_py.cymj' extension
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr/local
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr/local/lib
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-31

In [13]:
env.observation_space

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Box(-inf, inf, (11,), float64)

In [14]:
env.action_space

Box(-1.0, 1.0, (3,), float32)

In [15]:
obs_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]
timestep_limit = env.spec.max_episode_steps

# Loading base envs and base policies

In [16]:
import torch
from torch import distributions, nn

from pfrl import experiments, replay_buffers, utils
from pfrl.nn.lmbda import Lambda

In [18]:
# default params
policy_output_scale = 1.0

policy_hidden_dim = 256
policy_lr = 3e-4

q_func_hidden_dim = 256
q_func_lr = 3e-4

temperature_optimizer_lr = 3e-4

buffer_size = 10 ** 6
replay_start_size = 10 ** 4
batch_size = 256
gamma = 0.99

# steps = 10 ** 6
steps = 6 * 10 ** 5

In [19]:
def squashed_diagonal_gaussian_head(x):
    assert x.shape[-1] == action_size * 2
    mean, log_scale = torch.chunk(x, 2, dim=1)
    log_scale = torch.clamp(log_scale, -20.0, 2.0)
    var = torch.exp(log_scale * 2)
    base_distribution = distributions.Independent(
        distributions.Normal(loc=mean, scale=torch.sqrt(var)), 1
    )
    # cache_size=1 is required for numerical stability
    return distributions.transformed_distribution.TransformedDistribution(
        base_distribution, [distributions.transforms.TanhTransform(cache_size=1)]
    )

In [20]:
def make_policy_with_optimizer():
    policy = nn.Sequential(
        nn.Linear(obs_size, policy_hidden_dim),
        nn.ReLU(),
        nn.Linear(policy_hidden_dim, policy_hidden_dim),
        nn.ReLU(),
        nn.Linear(policy_hidden_dim, action_size * 2),
        Lambda(squashed_diagonal_gaussian_head),
    )
    torch.nn.init.xavier_uniform_(policy[0].weight)
    torch.nn.init.xavier_uniform_(policy[2].weight)
    torch.nn.init.xavier_uniform_(policy[4].weight, gain=policy_output_scale)
    policy_optimizer = torch.optim.Adam(policy.parameters(), lr=policy_lr)
    return policy, policy_optimizer

In [21]:
def make_q_func_with_optimizer():
    q_func = nn.Sequential(
        pfrl.nn.ConcatObsAndAction(),
        nn.Linear(obs_size + action_size, q_func_hidden_dim),
        nn.ReLU(),
        nn.Linear(q_func_hidden_dim, q_func_hidden_dim),
        nn.ReLU(),
        nn.Linear(q_func_hidden_dim, 1),
    )
    torch.nn.init.xavier_uniform_(q_func[1].weight)
    torch.nn.init.xavier_uniform_(q_func[3].weight)
    torch.nn.init.xavier_uniform_(q_func[5].weight)
    q_func_optimizer = torch.optim.Adam(q_func.parameters(), lr=q_func_lr)
    return q_func, q_func_optimizer

In [22]:
policy, policy_optimizer = make_policy_with_optimizer()
q_func1, q_func1_optimizer = make_q_func_with_optimizer()
q_func2, q_func2_optimizer = make_q_func_with_optimizer()

In [23]:
rbuffer = replay_buffers.ReplayBuffer(buffer_size)

In [24]:
agent = pfrl.agents.SoftActorCritic(
    policy,
    q_func1,
    q_func2,
    policy_optimizer,
    q_func1_optimizer,
    q_func2_optimizer,
    rbuffer,
    gamma=gamma,
    replay_start_size=replay_start_size,
    gpu=0 if torch.cuda.is_available() else -1,
    minibatch_size=batch_size,
    entropy_target=-action_size,
    temperature_optimizer_lr=temperature_optimizer_lr,
    act_deterministically=True,
)

In [25]:
base_policies = []
# for  key in ["standard", "cubblestone", "sandpaper"]:
for  key in ["standard", "cubblestone", "sandpaper"]:
    agent_ = deepcopy(agent)
    agent_.load(f"base_policies/{key}")
    base_policies.append(agent_)

In [26]:
base_envs = []
xml_files = ["hopper.xml", "envs/cobblestone_hopper_loose_v1.xml", "envs/sandpaper_hopper.xml"]
for i, xml in enumerate(xml_files):
    env_ = gym.make(env_name, exclude_current_positions_from_observation=False, xml_file=xml)
    base_envs.append(env_)

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment Hopper-v3 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/envs/mujoco/mujoco_env.py:237: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initi

In [27]:
base_policies

In [28]:
base_envs

[<TimeLimit<OrderEnforcing<StepAPICompatibility<PassiveEnvChecker<HopperEnv<Hopper-v3>>>>>>,
 <TimeLimit<OrderEnforcing<StepAPICompatibility<PassiveEnvChecker<HopperEnv<Hopper-v3>>>>>>,
 <TimeLimit<OrderEnforcing<StepAPICompatibility<PassiveEnvChecker<HopperEnv<Hopper-v3>>>>>>]

# Meta hopper env with terrain

In [29]:
def make_env(process_idx, test):
    env = gym.make(env_name)  #
    # env = gym.make(env_name, exclude_current_positions_from_observation=False)  #
    # Unwrap TimiLimit wrapper
    assert isinstance(env, gym.wrappers.TimeLimit)
    env = env.env
    # Use different random seeds for train and test envs
    process_seed = int(process_seeds[process_idx])
    env_seed = 2**32 - 1 - process_seed if test else process_seed
    env.seed(env_seed)
    # Cast observations to float32 because our model uses float32
    env = pfrl.wrappers.CastObservationToFloat32(env)
    # Normalize action space to [-1, 1]^n
    # env = pfrl.wrappers.NormalizeActionSpace(env)
    if monitor:
        env = gym.wrappers.Monitor(env, "monitor")
    if render:
        env = pfrl.wrappers.Render(env)
    return env

In [30]:
class HopperEnvWithTerrain(gym.Env):

    def __init__(
        self,
        base_envs: List[gym.Env] = base_envs,
        base_policies: Optional[List[pfrl.agent.Agent]] = base_policies,
        average_distance_of_each_terrain: float = 10.0,
        random_state: Optional[int] = None,
    ):
        if random_state is not None:
            self.random_ = check_random_state(random_state)
        else:
            self.random_ = check_random_state(12345)

        self.base_envs = base_envs
        self.base_policies = base_policies
        self.use_base_policy = (base_policies is not None)

        self.n_envs = len(base_envs)
        self.n_policies = len(base_policies)

        self.observation_space = self.base_envs[0].observation_space
        self.dim_observation = self.observation_space.shape[0]
        self.action_space = Discrete(3)

        self.env_change_position = self.random_.normal(
            loc=average_distance_of_each_terrain,
            scale=average_distance_of_each_terrain * 0.1,
            size=(10000, )
        ).cumsum()
        self.env_ids_position = self.random_.choice(
            self.n_policies,
            size=10000,
            replace=True,
        )

    def _get_env_id_given_position(self):
        env_id = np.argmax(self.env_change_position > self.position)
        env_id = self.env_ids_position[env_id]
        return env_id

    def _update_state_of_all_envs(
        self,
        target_env_id: int,
    ):
        current_env = self.base_envs[target_env_id]
        for i in range(self.n_envs):
            self.base_envs[i].data = current_env.data

    def reset(self):
        initial_states = self.base_envs[0].reset()

        self._update_state_of_all_envs(0)
        self.position = initial_states[-1]

        self.state = initial_states[:-1]

        return initial_states

    def step(self, action: int):
        env_id = self._get_env_id_given_position()

        if self.use_base_policy:
            # this is not working
            # with self.base_policies[action].eval_mode():
            action = self.base_policies[action].act(self.state)

        state, reward, done, info = self.base_envs[env_id].step(action)
        self._update_state_of_all_envs(env_id)

        self.state = state[:-1]

        return state, reward, done, info

    def render(self):
        env_id = self._get_env_id_given_position()
        return self.base_envs[env_id].render()


In [31]:
gym.register(
    id="Hopper-with-terrain-v3",
    entry_point=HopperEnvWithTerrain,
    max_episode_steps=timestep_limit,
)

## Initialize environment

In [32]:
env_name = "Hopper-with-terrain-v3"

In [33]:
env = make_env(0, test=False)

/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


In [34]:
def make_batch_env(test):
    return pfrl.envs.MultiprocessVectorEnv(
        [
            functools.partial(make_env, idx, test)
            for idx, env in enumerate(range(num_envs))
        ]
    )

## DDQN Config (with PFRL)

sample codes and parameters in [this file](https://github.com/pfnet/pfrl/blob/master/examples/quickstart/quickstart.ipynb).

In [35]:
# default params
buffer_size = 10 ** 6
epsilon = 0.3
gamma = 0.9

# steps = 10 ** 6
steps = 6 * 10 ** 5

In [36]:
class QFunction(torch.nn.Module):

    def __init__(self, obs_size, n_actions):
        super().__init__()
        self.l1 = torch.nn.Linear(obs_size, 50)
        self.l2 = torch.nn.Linear(50, 50)
        self.l3 = torch.nn.Linear(50, n_actions)

    def forward(self, x):
        h = x
        h = torch.nn.functional.relu(self.l1(h))
        h = torch.nn.functional.relu(self.l2(h))
        h = self.l3(h)
        return pfrl.action_value.DiscreteActionValue(h)

In [37]:
obs_size = env.observation_space.low.size
n_actions = env.action_space.n
q_func1 = QFunction(obs_size, n_actions)
optimizer = torch.optim.Adam(q_func1.parameters(), eps=1e-2)

In [38]:
q_func2 = torch.nn.Sequential(
    torch.nn.Linear(obs_size, 50),
    torch.nn.ReLU(),
    torch.nn.Linear(50, 50),
    torch.nn.ReLU(),
    torch.nn.Linear(50, n_actions),
    pfrl.q_functions.DiscreteActionValueHead(),
)


In [39]:
explorer = pfrl.explorers.ConstantEpsilonGreedy(
    epsilon=epsilon, random_action_func=env.action_space.sample
)
replay_buffer = pfrl.replay_buffers.ReplayBuffer(
    capacity=buffer_size,
)
phi = lambda x: x.astype(np.float32, copy=False)

## Train SAC policy online

In [40]:
agent = pfrl.agents.DoubleDQN(
    q_func1,
    optimizer,
    rbuffer,
    gamma,
    explorer,
    replay_start_size=500,
    update_interval=1,
    target_update_interval=100,
    phi=phi,
    gpu=0 if torch.cuda.is_available() else -1,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
agent

In [ ]:
experiments.train_agent_batch_with_evaluation(
    agent=agent,
    env=make_batch_env(test=False),
    eval_env=make_batch_env(test=False),
    outdir="logs/",
    steps=steps,
    eval_n_steps=None,
    eval_n_episodes=10,
    eval_interval=5 * 10 ** 3,
    log_interval=10 ** 3,
    max_episode_len=timestep_limit,
)

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be pas

KeyboardInterrupt: ignored

In [ ]:
agent.save("ddqn-default")

## online evaluation

In [ ]:
agent.load("ddqn-default")

In [ ]:
eval_stats = experiments.eval_performance(
    env=make_batch_env(test=True),
    agent=agent,
    n_steps=None,
    n_episodes=10,
    max_episode_len=timestep_limit,
)
print(
    "n_runs: {} mean: {} median: {} stdev {}".format(
        10,
        eval_stats["mean"],
        eval_stats["median"],
        eval_stats["stdev"],
    )
)

## render (TODO)

In [ ]:
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

In [ ]:
def make_env_for_render():
    env = gym.make(env_name, render_mode='rgb_array')
    env = gym.wrappers.RecordEpisodeStatistics(env)
    env.seed(seed)
    env.action_space.seed(seed)
    env.observation_space.seed(seed)
    return env

In [ ]:
env_ = make_env_for_render()

In [ ]:
plt.axis('off')
done = False
obs = env_.reset()

i = 0
while not done:
    i += 1
    if i % 20 == 0:
        ipythondisplay.clear_output(wait=True)
        # print("At timestep = ", i)
        screen = env_.render()
        plt.imshow(screen[0])
        ipythondisplay.display(plt.gcf())

    action = agent.act(obs)
    obs, reward, done, info = env_.step(action)

    if done:
        break